# Result Exploration
This notebook helps to build functions for printing results of CNN training. 

In [67]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from IPython.display import display
from IPython.display import display_html

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Scripts import Print_Functions as PF
from Scripts import Centralized_Pain_CNN as painCNN
from Scripts import Data_Loader_Functions as DL

FIGURES = os.path.join(module_path, "Figures")
RESULTS = os.path.join(module_path, "Results")
DATA = os.path.join(module_path, "Data", "Augmented Data", "Flexible Augmentation")

## Prepare Plots for Aggregate Metrics

In [2]:
# Define variables
model_type = 'Federated'
data_type = 'Unbalanced'

centralized_path = os.path.join(module_path, 'Models', 'Pain', 'Centralized', 'Final', data_type)
centralized_model_paths = [os.path.join(centralized_path, path) for path in os.listdir(centralized_path) if 'shard-0.00' in path]
federated_path = os.path.join(module_path, 'Models', 'Pain', 'Federated', 'Final', data_type)
federated_model_paths = [os.path.join(federated_path, path) for path in os.listdir(federated_path) if 'shard-0.00' in path]
model_paths = centralized_model_paths if model_type == 'Centralized' else federated_model_paths

### Step 0: Create Baseline Metrics

In [ ]:
# Load and prepare data
test_path = os.path.join(DATA, "group_2_test")
test_data, test_labels = DL.load_pain_data(test_path)
test_labels_binary = DL.reduce_pain_label_categories(test_labels[:, 4].astype(int), 1)
enc = OneHotEncoder(sparse=False, categories='auto')
test_labels_binary = enc.fit_transform(test_labels_binary.reshape(len(test_labels_binary), 1))
people = test_labels[:, 0].astype(int)

In [ ]:
# Load model
model = tf.keras.models.load_model(model_paths[0])

In [ ]:
# Make predictions
loss = tf.keras.losses.BinaryCrossentropy()
df = painCNN.evaluate_pain_cnn(model, 0, test_data, test_labels_binary, history=None, people=None, loss=loss)
df['Shard'] = 0.00

In [ ]:
# Save predictions to file
baseline_path = os.path.join(RESULTS, 'Final', data_type, '2019-07-31_PAIN_0-' + data_type.lower() + "-" + model_type + '-Baseline')
if not os.path.isdir(baseline_path):
    os.mkdir(baseline_path)

file = time.strftime("%Y-%m-%d-%H%M%S") + "_PAIN_" + data_type.lower() + "-" + model_type + ".csv"
df.to_csv(os.path.join(baseline_path, file))

### Step 1: Set parameters

In [3]:
# Set variable to plot
current_metric = 'Accuracy'
# current_metric = 'Precision'
# current_metric = 'Recall'
# current_metric = 'F1_Score'
# current_metric = 'Avg. Precision'

In [4]:
# Set experiment filter
criteria = '-unbalanced'

### Step 2: Get relevant folders

In [5]:
# Get all paths in Final Results folder
final_results = os.path.join(RESULTS, 'Final', data_type)
final_results_folders = sorted([os.path.join(final_results, path) for path in os.listdir(final_results)])

In [6]:
# Filter paths for relevant experiments
final_results_folders = [folder for folder in final_results_folders if criteria in folder]

### Step 3.1: Draw Aggregate plot

In [ ]:
metric = 'Aggregate ' + current_metric

In [ ]:
# Set PLT Parameters
plt.rcParams.update({'font.size': 18, 
                     'font.family' : 'arial', 
                     'font.weight' : 'normal',
                     'axes.titlesize' : 22})

# Draw Aggregate Plot
columns = ['Shard', 'Epoch', 'Loss', 'Aggregate Avg. Precision', 'Aggregate Accuracy', 
            'Aggregate Precision', 'Aggregate Recall', 'Aggregate F1_Score']

colors = ['#00065E', '#266DC9', '#F3752B', '#F02D3A', '#A20021']
line_style = ['--', '--', '-', '-', '-']

# Draw metrics
folders = [folder for folder in final_results_folders if 'Baseline' not in folder]
for idx, folder in enumerate(folders):
    df_concat = pd.DataFrame(columns=columns)
    for file in sorted(os.listdir(folder)):
        if 'shard-0.00' not in file:
            file = os.path.join(folder, file)
            df = pd.read_csv(file, index_col=0)
            df['Shard'] = float(file.split('_shard-')[1].split('_')[0])
            df = df[columns]
            df_concat = pd.concat((df_concat, df), ignore_index=True)
            df_concat.drop_duplicates(inplace=True)
    df_concat.reset_index(inplace=True)
    ax = df_concat[metric].plot(figsize=(24, 16), color=colors[idx], linewidth=1, linestyle=line_style[idx])

# Draw VLines
for i, file in enumerate(sorted(os.listdir(folders[0]))):
    ax.axvline(i * (df_concat['Epoch'].max() + 1), color='#C0C0C0', linestyle='dashed', label='_nolegend_')
    plt.text(i * (df_concat['Epoch'].max() + 1) + 1, 0.02, "Shard: {:.0%}".format(float(file.split('_shard-')[1].split('_')[0])))

# Draw Baselines
baseline_folders = sorted([folder for folder in final_results_folders if 'Baseline' in folder])
h_colors =['#002BA0', '#DD0426']
h_line_style = ['--', '-']
for idx, folder in enumerate(baseline_folders):
    for file in os.listdir(folder):
        file = os.path.join(folder, file)
        df = pd.read_csv(file, index_col=0)
        ax.axhline(df[metric].iloc[0], color=h_colors[idx], linestyle=h_line_style[idx])


# Set plot parameters
folders.extend(baseline_folders)
ax.legend([folder.split('PAIN_')[1] for folder in folders], loc='best')
ax.set_title("{} | All Experiments".format(metric))
ax.set_yticks(np.arange(0,1.1,0.1))
ax.set_yticklabels(['{:,.0%}'.format(x) for x in np.arange(0,1.1,0.1)])
ax.set_ylabel(metric)
ax.set_xlabel('Epochs / Communication Rounds')

In [ ]:
# Save figure
fig_path = os.path.join(FIGURES, 'Final', data_type)
file = os.path.join(fig_path, metric + '.png')
fig = ax.get_figure()
fig.savefig(file, dpi=300)

### Step 3.2 Individual Standard Deviation

In [ ]:
# Specify Metrics
metric = 'Individual ' + current_metric

In [ ]:
# Set PLT Parameters
plt.rcParams.update({'font.size': 18, 
                     'font.family' : 'arial', 
                     'font.weight' : 'normal',
                     'axes.titlesize' : 22})

# Draw standard deviation
columns = ['Shard', 'Epoch', 'Person', 'Loss', 'Individual Avg. Precision', 'Individual Accuracy', 
            'Individual Precision', 'Individual Recall', 'Individual F1-Score']

colors = ['#00065E', '#266DC9', '#F3752B', '#F02D3A', '#A20021']
line_style = ['--', '--', '-', '-', '-']

# Plot standard deviations
folders = [folder for folder in final_results_folders if 'Baseline' not in folder]
for idx, folder in enumerate(folders):
    df_concat = None
    for file in sorted(os.listdir(folder)):
        if 'shard-0.00' not in file:
            file = os.path.join(folder, file)
            df = pd.read_csv(file, index_col=0)
            df['Shard'] = float(file.split('_shard-')[1].split('_')[0])
            df = df[columns]
            df['Person'] = df['Person'].astype(int)
            pivot = df.pivot(index='Epoch', columns='Person', values=metric)
            if df_concat is None:
                df_concat = pd.DataFrame(columns=pivot.columns)
            df_concat = pd.concat((df_concat, pivot), ignore_index=True)
    df_concat['Mean'] = df_concat.mean(axis=1)
    df_concat['SD'] = df_concat.std(axis=1)
    ax = df_concat['SD'].plot(figsize=(24, 16), color=colors[idx], linewidth=1, linestyle=line_style[idx])
    
for i, file in enumerate(sorted(os.listdir(folders[0]))):
    ax.axvline(i * int(len(df_concat) / len(os.listdir(folders[0]))), color='#C0C0C0', linestyle='dashed', label='_nolegend_')
    plt.text(i * int(len(df_concat) / len(os.listdir(folders[0]))) + 1, 0.005, "Shard: {:.0%}".format(float(file.split('_shard-')[1].split('_')[0])))
    
ax.legend([folder.split('PAIN_')[1] for folder in folders], loc='best')
ax.set_title("{} | Standard deviation between subjects | All Experiments".format(metric))
ax.set_yticks(np.arange(0, 0.5, 0.1))
ax.set_ylabel('Standard deviation between subjects')
ax.set_xlabel('Epochs / Communication Rounds')
metric = 'SD_' + metric

In [ ]:
# Save figure
fig_path = os.path.join(FIGURES, 'Final', data_type)
file = os.path.join(fig_path, metric + '.png')
fig = ax.get_figure()
fig.savefig(file, dpi=300)

### Step 3.3 Individual metrics

In [ ]:
# Specify Metrics
metric = 'Individual ' + current_metric

In [ ]:
# Set PLT Parameters
plt.rcParams.update({'font.size': 12, 
                     'font.family' : 'arial', 
                     'font.weight' : 'normal',
                     'axes.titlesize' : 14})

# Draw individual metrics
columns = ['Shard', 'Epoch', 'Person', 'Loss', 'Individual Avg. Precision', 'Individual Accuracy', 
            'Individual Precision', 'Individual Recall', 'Individual F1-Score']

colors = ['#00065E', '#266DC9', '#F3752B', '#F02D3A', '#A20021']
line_style = ['--', '--', '-', '-', '-']

# Plot standard deviations
fig = plt.figure(dpi=300, figsize=(48, 60))
folders = [folder for folder in final_results_folders if 'Baseline' not in folder]
for idx, folder in enumerate(folders):
    df_concat = None
    for file in sorted(os.listdir(folder)):
        if 'shard-0.00' not in file:
            file = os.path.join(folder, file)
            df = pd.read_csv(file, index_col=0)
            df['Shard'] = float(file.split('_shard-')[1].split('_')[0])
            df = df[columns]
            df['Person'] = df['Person'].astype(int)
            pivot = df.pivot(index='Epoch', columns='Person', values=metric)
            if df_concat is None:
                df_concat = pd.DataFrame(columns=pivot.columns)
            df_concat = pd.concat((df_concat, pivot), ignore_index=True)
    ax = df_concat.plot(figsize=(24, 20), linewidth=1, ax=fig.add_subplot(3, 2, idx+1))
    
    for i, file in enumerate(sorted(os.listdir(folders[0]))):
        ax.axvline(i * int(len(df_concat) / len(os.listdir(folders[0]))), color='#C0C0C0', linestyle='dashed', label='_nolegend_')
        plt.text(i * int(len(df_concat) / len(os.listdir(folders[0]))) + 1, 0.005, "Shard: {:.0%}".format(float(file.split('_shard-')[1].split('_')[0])))
    
    ax.legend(loc='best', title='Test Subject')
    ax.set_title("{}".format(folder.split('PAIN_')[1]))
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in np.arange(0,1.1,0.1)])
    ax.set_ylabel(metric)
    ax.set_xlabel('Epochs / Communication Rounds')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle(metric, fontsize=20)
plt.show()

In [ ]:
# Save figure
fig_path = os.path.join(FIGURES, 'Final', data_type)
file = os.path.join(fig_path, metric + '.png')
fig = ax.get_figure()
fig.savefig(file, dpi=300)

### Step 3.4 Individual Metrics - Final Epoch

In [ ]:
# Specify Metrics
current_metric = 'Avg. Precision'
metric = 'Individual ' + current_metric

In [ ]:
# Set PLT Parameters
plt.rcParams.update({'font.size': 12, 
                     'font.family' : 'arial', 
                     'font.weight' : 'normal',
                     'axes.titlesize' : 14})

# Draw individual metrics
columns = ['Shard', 'Epoch', 'Person', 'Loss', 'Individual Avg. Precision', 'Individual Accuracy', 
            'Individual Precision', 'Individual Recall', 'Individual F1-Score']

colors = ['#00065E', '#266DC9', '#F3752B', '#F02D3A', '#A20021']
line_style = ['--', '--', '-', '-', '-']

# Plot standard deviations
fig = plt.figure(dpi=300, figsize=(48, 60))
folders = [folder for folder in final_results_folders if 'Baseline' not in folder]
for idx, folder in enumerate(folders):
    df_concat = None
    for file in sorted(os.listdir(folder)):
        if 'shard-0.00' not in file:
            file = os.path.join(folder, file)
            df = pd.read_csv(file, index_col=0)
            df['Shard'] = float(file.split('_shard-')[1].split('_')[0])
            df = df[columns]
            df['Person'] = df['Person'].astype(int)
            df = df[df['Epoch'] == df['Epoch'].max()]
            if df_concat is None:
                df_concat = pd.DataFrame(columns=df.columns)
            df_concat = pd.concat((df_concat, df), ignore_index=True)
    pivot = df_concat.pivot(index='Shard', columns='Person', values=metric)
    ax = pivot.plot(figsize=(24, 20), linewidth=1, ax=fig.add_subplot(3, 2, idx+1))
    ax.legend(loc='best', title='Test Subject')
    ax.set_title("{}".format(folder.split('PAIN_')[1]))
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in np.arange(0,1.1,0.1)])
    ax.set_ylabel(metric)
    ax.set_xticks(pivot.index)
    ax.set_xticklabels(['{:,.0%}'.format(x) for x in pivot.index])
    ax.set_xlabel('Shard')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("Last Epoch Value: " + metric, fontsize=20)
plt.show()

In [ ]:
# Save figure
fig_path = os.path.join(FIGURES, 'Final', data_type)
file = os.path.join(fig_path, "Last Epoch " + metric + '.png')
fig = ax.get_figure()
fig.savefig(file, dpi=300)

## Step 4: Sumarize data

In [75]:
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.render()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [80]:
# Display Data in Dataframe
metrics = ['Individual Accuracy', 'Individual Precision', 'Individual Recall', 'Individual F1-Score']
display_type = 'single'

folders = [folder for folder in final_results_folders if 'Baseline' not in folder]
for idx, folder in enumerate(folders):
    df_concat = None
    pivots = []
    print("\n\n\033[1m{}\033[0m".format(folder.split('PAIN_')[1]))
    for file in sorted(os.listdir(folder)):
        if 'shard-0.00' not in file:
            file = os.path.join(folder, file)
            df = pd.read_csv(file, index_col=0)
            df['Shard'] = float(file.split('_shard-')[1].split('_')[0])
            df['Person'] = df['Person'].astype(int)
            if df_concat is None:
                df_concat = pd.DataFrame(columns=df.columns)
            df_concat = pd.concat((df_concat, df), ignore_index=True)
            df_concat.drop_duplicates(inplace=True)
    df_concat.reset_index(inplace=True)
    for metric in metrics:
        columns = 'Shard'
        index = 'Person'
        df_filtered = df_concat[df_concat['Epoch'] == df_concat['Epoch'].max()]
        pivot = df_filtered[[index, columns, metric]].drop_duplicates().pivot(index=index, columns=columns, values=metric)
        cm = sns.diverging_palette(10, 130, n=9, as_cmap=True)
        style = pivot.style.background_gradient(cmap=cm).format('{:,.0%}').set_caption(metric)
        pivots.append(style)
    display_side_by_side(pivots[0], pivots[1])
    display_side_by_side(pivots[2], pivots[3])



1-unbalanced-Centralized-no-pre-training


Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,79%,91%,93%,89%,94%,92%,91%,93%
48,50%,67%,70%,78%,87%,86%,86%,88%
52,91%,92%,92%,92%,92%,94%,95%,95%
59,66%,72%,69%,73%,81%,80%,77%,80%
64,83%,77%,84%,85%,81%,86%,84%,86%
80,52%,73%,76%,73%,74%,75%,76%,77%
92,39%,48%,71%,75%,79%,77%,81%,85%
96,78%,89%,88%,87%,84%,89%,89%,89%
107,72%,81%,87%,90%,94%,91%,90%,92%


Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,51%,77%,74%,77%,64%,72%,67%,74%
48,68%,62%,76%,65%,50%,65%,71%,71%
52,44%,56%,47%,61%,61%,58%,61%,64%
59,16%,85%,96%,95%,91%,91%,96%,96%
64,56%,53%,58%,73%,85%,85%,89%,91%
80,90%,94%,93%,92%,95%,95%,96%,97%
92,96%,93%,96%,96%,96%,99%,97%,96%
96,70%,52%,56%,70%,74%,74%,74%,71%
107,83%,84%,81%,93%,89%,95%,95%,95%




2-unbalanced-Centralized-pre-training


Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,71%,86%,88%,89%,89%,92%,93%,92%
48,65%,75%,81%,78%,76%,86%,87%,87%
52,96%,92%,91%,94%,93%,93%,93%,92%
59,39%,53%,60%,69%,71%,73%,71%,73%
64,67%,82%,83%,83%,84%,84%,83%,86%
80,57%,62%,68%,72%,72%,75%,75%,75%
92,35%,53%,65%,72%,72%,75%,75%,80%
96,91%,92%,89%,89%,88%,88%,87%,90%
107,64%,84%,85%,88%,87%,90%,91%,91%


...

Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,85%,80%,83%,83%,86%,89%,90%,90%
48,76%,73%,74%,76%,79%,84%,80%,83%
52,96%,95%,92%,92%,92%,94%,93%,94%
59,54%,63%,65%,64%,69%,71%,69%,72%
64,70%,76%,79%,77%,78%,81%,83%,84%
80,54%,66%,67%,66%,68%,70%,73%,73%
92,55%,53%,61%,62%,68%,72%,76%,79%
96,91%,90%,85%,83%,83%,86%,88%,89%
107,75%,76%,78%,83%,84%,87%,88%,90%


Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,21%,26%,28%,54%,62%,69%,69%,69%
48,15%,29%,50%,53%,59%,56%,62%,68%
52,33%,39%,44%,61%,67%,67%,69%,67%
59,56%,58%,75%,84%,80%,87%,78%,80%
64,64%,55%,60%,71%,80%,80%,80%,82%
80,71%,76%,84%,90%,93%,94%,93%,95%
92,42%,67%,80%,86%,91%,93%,94%,94%
96,5%,24%,36%,62%,60%,74%,72%,74%
107,41%,58%,69%,82%,90%,92%,91%,95%




5-unbalanced-Federated-federated-pre-training


Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,91%,93%,92%,93%,94%,93%,95%,94%
48,86%,88%,82%,86%,86%,87%,87%,88%
52,97%,97%,95%,95%,96%,95%,95%,94%
59,67%,82%,86%,83%,82%,81%,84%,85%
64,88%,93%,82%,88%,86%,87%,89%,90%
80,56%,75%,70%,75%,75%,78%,77%,77%
92,56%,72%,71%,80%,80%,81%,83%,84%
96,90%,92%,91%,91%,91%,91%,91%,92%
107,77%,88%,82%,91%,93%,93%,94%,94%


Shard,0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6
Person,,,,,,,,
43,26%,59%,74%,69%,77%,77%,85%,77%
48,41%,38%,53%,68%,68%,79%,79%,79%
52,42%,47%,61%,53%,58%,61%,61%,61%
59,44%,80%,95%,93%,95%,95%,93%,95%
64,53%,56%,75%,75%,84%,87%,91%,91%
80,85%,90%,96%,95%,96%,96%,96%,98%
92,69%,80%,89%,94%,96%,96%,96%,96%
96,5%,18%,59%,65%,76%,79%,80%,79%
107,56%,74%,87%,93%,94%,93%,95%,94%


In [83]:
type(3) is int

True